In [65]:
from load_data import load_data

train_data, val_data, test_data = load_data('RAF_DB', rgb=True, image_size=(100,100))

Found 11040 files belonging to 7 classes.
Found 1231 files belonging to 7 classes.
Found 3068 files belonging to 7 classes.


In [4]:
# Do zrobienia - puszczenie jeszcze jakiegos keras_tune, ale moze troszke bardziej sensownego i sprobowanie lekkiego stunowania strukturki
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
import tensorflow as tf 

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))


In [5]:
# Model training
from keras.callbacks import EarlyStopping, ModelCheckpoint
from config import BATCH_SIZE, EPOCHS
from keras.callbacks import ReduceLROnPlateau

best_model_path = 'models/best_fer_no_aug.h5'

earlystop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history = model.fit(train_data, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data=val_data, callbacks = [earlystop, learning_rate_reduction])

model.save_weights(best_model_path)

Epoch 1/50


2024-04-28 16:31:32.001993: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout_4/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-04-28 16:31:33.692325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2024-04-28 16:31:34.855482: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-04-28 16:31:34.859194: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f344f6ef2a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-28 16:31:34.859225: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-04-28 16:31:34.863181: I tensorflow/compiler/mlir/tensorflow/util

404/404 [==============================] - 17s 26ms/step - loss: 2.0204 - accuracy: 0.3132 - val_loss: 1.5340 - val_accuracy: 0.4168 - lr: 0.0010
Epoch 2/50
404/404 [==============================] - 10s 25ms/step - loss: 1.4798 - accuracy: 0.4516 - val_loss: 1.3846 - val_accuracy: 0.4736 - lr: 0.0010
Epoch 3/50
404/404 [==============================] - 10s 25ms/step - loss: 1.3103 - accuracy: 0.5033 - val_loss: 1.2953 - val_accuracy: 0.5118 - lr: 0.0010
Epoch 4/50
404/404 [==============================] - 10s 25ms/step - loss: 1.2143 - accuracy: 0.5382 - val_loss: 1.1776 - val_accuracy: 0.5623 - lr: 0.0010
Epoch 5/50
404/404 [==============================] - 10s 25ms/step - loss: 1.1424 - accuracy: 0.5681 - val_loss: 1.2098 - val_accuracy: 0.5567 - lr: 0.0010
Epoch 6/50
404/404 [==============================] - 10s 25ms/step - loss: 1.0893 - accuracy: 0.5910 - val_loss: 1.1194 - val_accuracy: 0.5825 - lr: 0.0010
Epoch 7/50
404/404 [==============================] - 10s 25ms/step -

In [62]:
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#model.load_weights('models/unkown_best_fer_no_aug.h5')
from data_set_utils import TestDataMetrics
tdm = TestDataMetrics(test_data, model)
tdm.print_confusion_matrix()
model.evaluate(test_data)

ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.8/dist-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequentia

-----------------------------------------------------------------
GRID SEARCH BASED ON INITIAL NETWORK, BASED ON WHITCH THE MODEL ABOVE HAVE BEEN TWEAKED:

In [6]:
from keras_tuner import HyperModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        # Dodaj warstwy konwolucyjne z możliwością wyboru liczby filtrów
        for i in range(hp.Int('conv_blocks', 3, 5, default=4)):
            filters = hp.Choice(f'filters_{i}', [32, 64, 128, 256])
            activation = hp.Choice('activation', ['relu', 'softmax'])
            for _ in range(2):  # Dwie warstwy konwolucyjne na blok
                model.add(Conv2D(filters=filters, kernel_size=(3, 3), activation=activation, padding='same'))
                model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(rate=hp.Float('dropout_conv', min_value=0.2, max_value=0.3, step=0.05)))

        model.add(Flatten())
        # Warstwa gęsta na końcu
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout_dense', min_value=0.3, max_value=0.5, step=0.05)))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model


In [7]:
from keras_tuner import RandomSearch

# Ustawienie rozmiaru wejścia i liczby klas
hypermodel = CNNHyperModel(input_shape=(100, 100, 1), num_classes=7)

tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='trial_dir',
    project_name='hparam_tuning'
)

tuner.search(train_data,
             epochs=10, 
             validation_data=val_data
            )

# Pobranie najlepszego modelu
best_model = tuner.get_best_models(num_models=1)[0]

Reloading Tuner from trial_dir/hparam_tuning/tuner0.json


---------------------------------------------------
BASE RESNET50 code

In [63]:
from tensorflow.keras.applications import ResNet50, ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


ResNet = ResNet50V2(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
ResNet.trainable = True

model = Sequential([
    ResNet,
    Flatten(),
    Dense(7, activation='softmax'),
])

In [64]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from config import BATCH_SIZE, EPOCHS
from keras.callbacks import ReduceLROnPlateau

best_model_path = 'models/base_flaten_res_raf.h5'

earlystop = EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True, verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.5, min_lr=0.0002)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

train_history = model.fit(train_data, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data=val_data, callbacks = [earlystop, learning_rate_reduction])

model.save_weights(best_model_path)

Epoch 1/50
173/173 [==============================] - 369s 2s/step - loss: 1.1481 - accuracy: 0.6195 - val_loss: 6.9572 - val_accuracy: 0.6434 - lr: 0.0010
Epoch 2/50
173/173 [==============================] - 310s 2s/step - loss: 0.6776 - accuracy: 0.7607 - val_loss: 0.8977 - val_accuracy: 0.7132 - lr: 0.0010
Epoch 3/50
173/173 [==============================] - 307s 2s/step - loss: 0.4882 - accuracy: 0.8287 - val_loss: 1.0788 - val_accuracy: 0.6742 - lr: 0.0010
Epoch 4/50
173/173 [==============================] - ETA: 0s - loss: 0.7205 - accuracy: 0.7579
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
173/173 [==============================] - 307s 2s/step - loss: 0.7205 - accuracy: 0.7579 - val_loss: 7132.3608 - val_accuracy: 0.3875 - lr: 0.0010
Epoch 5/50
173/173 [==============================] - 312s 2s/step - loss: 0.7952 - accuracy: 0.7478 - val_loss: 0.8696 - val_accuracy: 0.7124 - lr: 5.0000e-04
Epoch 6/50
173/173 [==============================] 

In [ ]:
from data_set_utils import plot_loss_and_acc
plot_loss_and_acc(train_history)

-----------------------------------------------
TUNED RESNET50 CODE

In [ ]:
ResNet = ResNet50V2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
ResNet.trainable = True

for layer in ResNet.layers[:-50]:
    layer.trainable = False
    
model = Sequential([
    ResNet,
    Dropout(.25),
    BatchNormalization(),
    Flatten(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(.5),
    Dense(7,activation='softmax')
])

In [ ]:
from 